## Baseline Model
- for 'sex' and 'race'

In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.insert(0, project_root)

In [6]:
# Imports 
import pandas as pd

# Internal Functions
from src.data_preprocessing import preprocessing_adult

# pd settings
pd.set_option('display.max_columns', None)

# data
from aif360.datasets import AdultDataset, BinaryLabelDataset

# preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# model
from sklearn.linear_model import LogisticRegression

# Evaluation
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

In [7]:
# 1) Retrieve data
protected = 'sex'
privileged_value   = 1.0
unprivileged_value = 0.0

ad = AdultDataset(
    protected_attribute_names=[protected],
    privileged_classes=[['Male']],
    categorical_features=[
        'workclass',
        'education',
        'marital-status',
        'occupation',
        'relationship',
        'race',
        'native-country'
    ],
    features_to_drop=[''],
    instance_weights_name='fnlwgt',
    custom_preprocessing=preprocessing_adult,
    na_values=[]
)
# 2) Build a single DataFrame (features + label) 
df = pd.DataFrame(ad.features, columns=ad.feature_names)
df['label'] = ad.labels.ravel()

# 3) Stratified splitter for 25 runs, 80/20 split
sss = StratifiedShuffleSplit(n_splits=25, test_size=0.2, random_state=42)

results = []
for train_idx, test_idx in sss.split(df, df['label']):
    train_df = df.iloc[train_idx]
    test_df  = df.iloc[test_idx]

    # 4) Scale features
    scaler = StandardScaler().fit(train_df[ad.feature_names])
    X_train = scaler.transform(train_df[ad.feature_names])
    X_test  = scaler.transform(test_df[ad.feature_names])
    y_train = train_df['label']
    y_test  = test_df['label']

    # 5) Train & predict
    lr = LogisticRegression(solver='liblinear')
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)

    # 6) Wrap into AIF360 datasets
    test_orig = BinaryLabelDataset(
        favorable_label=1.0, unfavorable_label=0.0,
        df=test_df,
        label_names=['label'],
        protected_attribute_names=[protected],
        privileged_protected_attributes=[[privileged_value]],
        unprivileged_protected_attributes=[[unprivileged_value]]
    )
    test_pred = test_orig.copy(deepcopy=True)
    test_pred.labels = y_pred.reshape(-1,1)

    # 7a) Outcome‐level metrics
    bldm = BinaryLabelDatasetMetric(
        test_pred,
        privileged_groups=[{protected: privileged_value}],
        unprivileged_groups=[{protected: unprivileged_value}]
    )
    spd = bldm.statistical_parity_difference()
    di  = bldm.disparate_impact()

    # 7b) Error-Based metrics
    cls = ClassificationMetric(
        test_orig, test_pred,
        privileged_groups=[{protected: privileged_value}],
        unprivileged_groups=[{protected: unprivileged_value}]
    )
    eod = cls.equal_opportunity_difference()
    aod = cls.average_odds_difference()

    # 9) Compute performance metrics
    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred)

    # 10) Append to results
    results.append({
        'accuracy': acc,
        'f1_score': f1,
        'statistical_parity_difference': spd,
        'disparate_impact': di,
        'equal_opportunity_difference': eod,
        'average_odds_difference': aod
    })

# 11) Aggregate results
metrics_df = pd.DataFrame(results)
metrics_df_agg = metrics_df.agg(['mean', 'std'])

In [9]:
metrics_df_agg

,accuracy,f1_score,statistical_parity_difference,disparate_impact,equal_opportunity_difference,average_odds_difference
mean,0.850875,0.657510,-0.182034,0.290624,-0.119800,-0.099392
std,0.002878,0.005878,0.006366,0.016677,0.025528,0.013348


In [10]:
# 1) Retrieve data
protected = 'race'
privileged_value   = 1.0
unprivileged_value = 0.0

ad = AdultDataset(
    protected_attribute_names=[protected],
    privileged_classes=[['White']],
    categorical_features=[
        'workclass',
        'education',
        'marital-status',
        'occupation',
        'relationship',
        'sex',
        'native-country'
    ],
    features_to_drop=[''],
    instance_weights_name='fnlwgt',
    custom_preprocessing=preprocessing_adult,
    na_values=[]
)
# 2) Build a single DataFrame (features + label) 
df = pd.DataFrame(ad.features, columns=ad.feature_names)
df['label'] = ad.labels.ravel()

# 3) Stratified splitter for 25 runs, 80/20 split
sss = StratifiedShuffleSplit(n_splits=25, test_size=0.2, random_state=42)

results = []
for train_idx, test_idx in sss.split(df, df['label']):
    train_df = df.iloc[train_idx]
    test_df  = df.iloc[test_idx]

    # 4) Scale features
    scaler = StandardScaler().fit(train_df[ad.feature_names])
    X_train = scaler.transform(train_df[ad.feature_names])
    X_test  = scaler.transform(test_df[ad.feature_names])
    y_train = train_df['label']
    y_test  = test_df['label']

    # 5) Train & predict
    lr = LogisticRegression(solver='liblinear')
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)

    # 6) Wrap into AIF360 datasets
    test_orig = BinaryLabelDataset(
        favorable_label=1.0, unfavorable_label=0.0,
        df=test_df,
        label_names=['label'],
        protected_attribute_names=[protected],
        privileged_protected_attributes=[[privileged_value]],
        unprivileged_protected_attributes=[[unprivileged_value]]
    )
    test_pred = test_orig.copy(deepcopy=True)
    test_pred.labels = y_pred.reshape(-1,1)

    # 7a) Outcome‐level metrics
    bldm = BinaryLabelDatasetMetric(
        test_pred,
        privileged_groups=[{protected: privileged_value}],
        unprivileged_groups=[{protected: unprivileged_value}]
    )
    spd = bldm.statistical_parity_difference()
    di  = bldm.disparate_impact()

    # 7b) Error-Based metrics
    cls = ClassificationMetric(
        test_orig, test_pred,
        privileged_groups=[{protected: privileged_value}],
        unprivileged_groups=[{protected: unprivileged_value}]
    )
    eod = cls.equal_opportunity_difference()
    aod = cls.average_odds_difference()

    # 9) Compute performance metrics
    acc = accuracy_score(y_test, y_pred)
    f1  = f1_score(y_test, y_pred)

    # 10) Append to results
    results.append({
        'accuracy': acc,
        'f1_score': f1,
        'statistical_parity_difference': spd,
        'disparate_impact': di,
        'equal_opportunity_difference': eod,
        'average_odds_difference': aod
    })

# 11) Aggregate results
metrics_df = pd.DataFrame(results)
metrics_df_agg = metrics_df.agg(['mean', 'std'])

In [11]:
metrics_df_agg

,accuracy,f1_score,statistical_parity_difference,disparate_impact,equal_opportunity_difference,average_odds_difference
mean,0.850871,0.657504,-0.095198,0.546383,-0.080326,-0.057502
std,0.002882,0.005880,0.005256,0.023411,0.023949,0.012064
